### Approach for Cyclic Graphs

In [16]:
# import math
# from collections import defaultdict
# import csv

# class Label:
#     def __init__(self, node, reward, penalty, visited, predecessor=None, last_edge=None):
#         self.node = node
#         self.reward = reward
#         self.penalty = penalty
#         self.visited = visited          # frozenset for hashability
#         self.predecessor = predecessor  # Link to previous label

#     def __repr__(self):
#         return f"Label({self.node}, R={self.reward}, L={self.penalty})"

# class Graph:
#     def __init__(self):
#         self.edges = defaultdict(list)  # {u: [(v, reward, penalty)]}

# def load_graph_from_csv(filename):
#     G = Graph()
#     with open(filename, "r") as f:
#         reader = csv.reader(f)
#         next(reader)  # Skip header
#         for u, v, weight in reader:
#             weight = int(weight)
#             reward = weight if weight > 0 else 0
#             penalty = -weight if weight < 0 else 0
#             G.edges[u].append((v, reward, penalty))
#     return G

# def is_dominated(new_label, existing_labels):
#     """Check if new_label is dominated by any existing label."""
#     for label in existing_labels:
#         if (label.reward >= new_label.reward and label.penalty <= new_label.penalty):
#             return True
#     return False

# def rrp_fptas(G, s, t, C, epsilon):
#     """SSMOSP-inspired FPTAS for RRP with cycle prevention."""
#     nodes = list(G.edges.keys())
#     n = len(nodes)
#     if n == 0:
#         return 0, 0, []
#     delta = epsilon / (n - 1) if n > 1 else epsilon

#     # Initialize labels: {node: {interval: list of non-dominated labels}}
#     labels = defaultdict(lambda: defaultdict(list))
#     initial_visited = frozenset([s])
#     labels[s][0].append(Label(s, 0, 0, initial_visited))

#     for _ in range(n-1):  # Bellman-Ford iterations
#         new_labels = defaultdict(lambda: defaultdict(list))
#         for u in labels:
#             for interval in labels[u]:
#                 for label_u in labels[u][interval]:
#                     # Skip processing edges if current node is the target
#                     if label_u.node == t:
#                         continue  # Do not extend paths beyond t

#                     for (v, r, l) in G.edges[label_u.node]:
#                         # Check for cycles
#                         if v in label_u.visited:
#                             continue

#                         # Compute new reward and penalty
#                         R_new = label_u.reward + r
#                         L_new = label_u.penalty + l

#                         if L_new > C:
#                             continue

#                         # Compute interval for R_new
#                         if R_new <= 0:
#                             new_interval = 0
#                         else:
#                             new_interval = max(0, int(math.log(R_new, 1 + delta)))

#                         # Create new visited set
#                         new_visited = label_u.visited.union(frozenset([v]))

#                         # Create new label
#                         new_label = Label(
#                             node=v,
#                             reward=R_new,
#                             penalty=L_new,
#                             visited=new_visited,
#                             predecessor=label_u
#                         )

#                         # Check dominance before adding
#                         existing_labels = new_labels[v][new_interval]
#                         if not is_dominated(new_label, existing_labels):
#                             # Remove labels dominated by new_label
#                             filtered = [label for label in existing_labels 
#                                         if not (new_label.reward >= label.reward and new_label.penalty <= label.penalty)]
#                             filtered.append(new_label)
#                             new_labels[v][new_interval] = filtered

#         # Merge new labels into the main labels
#         for v in new_labels:
#             for interval in new_labels[v]:
#                 combined = labels[v][interval] + new_labels[v][interval]
#                 # Filter dominated labels
#                 non_dominated = []
#                 for label in combined:
#                     if not is_dominated(label, non_dominated):
#                         # Remove dominated labels
#                         non_dominated = [l for l in non_dominated if not (label.reward >= l.reward and label.penalty <= l.penalty)]
#                         non_dominated.append(label)
#                 labels[v][interval] = non_dominated

#     # Extract the best path to t
#     best_reward = 0
#     best_label = None
#     for interval in labels.get(t, {}):
#         for label in labels[t][interval]:
#             if label.penalty <= C and label.reward > best_reward:
#                 best_reward = label.reward
#                 best_label = label

#     # Reconstruct path if a valid label is found
#     if best_label is None:
#         return 0, 0, []

#     path = []
#     current = best_label
#     while current:
#         path.append(current.node)
#         current = current.predecessor
#     path.reverse()

#     return best_reward, best_label.penalty, path

# # Usage Example
# G = load_graph_from_csv("graph_data.csv")
# max_reward, total_penalty, path = rrp_fptas(G, "n0", f"n{len(G.edges)-1}", C=20, epsilon=0.2)

# if path:
#     print(f"Path: {' -> '.join(path)}")
#     print(f"Reward: {max_reward}, Penalty: {total_penalty}")
# else:
#     print("No valid path found.")

In [3]:
import random
import csv
import math
from collections import defaultdict, deque



def load_graph_from_csv(filename):
    graph = defaultdict(dict)
    
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header
        
        for row in reader:
            source, target, weight = row
            weight = int(weight)
            graph[source][target] = weight
    
    return graph


class FPTAS_RRP:
    def __init__(self, graph, source, target, constraint_C, epsilon):
        self.graph = graph
        self.source = source
        self.target = target
        self.C = constraint_C
        self.epsilon = epsilon
        self.n = len(graph)
        self.delta = epsilon / (self.n - 1)
        
        # Transform the graph to bi-objective format
        self.bi_graph = self.transform_graph()
    
    def transform_graph(self):
        bi_graph = defaultdict(dict)
        for u in self.graph:
            for v, weight in self.graph[u].items():
                if weight > 0:
                    bi_graph[u][v] = (weight, 0)  # Reward
                else:
                    bi_graph[u][v] = (0, abs(weight))  # Penalty
        return bi_graph
    
    def get_bucket(self, reward):
        if reward <= 0:
            return 0
        return math.floor(math.log(reward, 1 + self.delta))
    
    def run(self):
        # Dictionary to store paths for each node and bucket
        pareto_sets = {node: {} for node in self.graph}
        
        # Initialize the source node
        pareto_sets[self.source][0] = (0, 0, [self.source])
        
        # Queue for nodes to process
        queue = deque([self.source])
        in_queue = {self.source}
        
        while queue:
            node = queue.popleft()
            in_queue.remove(node)
            
            # Process each label at the current node
            for bucket, (reward, penalty, path) in list(pareto_sets[node].items()):
                # Process each neighbor
                for neighbor, (edge_reward, edge_penalty) in self.bi_graph[node].items():
                    # Skip if the neighbor is already in the path (avoid cycles)
                    if neighbor in path:
                        continue
                    
                    # Calculate new reward and penalty
                    new_reward = reward + edge_reward
                    new_penalty = penalty + edge_penalty
                    
                    # Skip if the new penalty exceeds the constraint
                    if new_penalty > self.C:
                        continue
                    
                    # Get the bucket for the new reward
                    new_bucket = self.get_bucket(new_reward)
                    
                    # Check if we already have a path for this bucket
                    is_dominated = False
                    
                    if new_bucket in pareto_sets[neighbor]:
                        _, old_penalty, _ = pareto_sets[neighbor][new_bucket]
                        if old_penalty <= new_penalty:
                            is_dominated = True
                    
                    if not is_dominated:
                        # Update with the new path
                        new_path = path + [neighbor]
                        pareto_sets[neighbor][new_bucket] = (new_reward, new_penalty, new_path)
                        
                        # Add the neighbor to the queue for processing
                        if neighbor not in in_queue:
                            queue.append(neighbor)
                            in_queue.add(neighbor)
        
        # Find the best path to the target that satisfies the constraint
        best_reward = 0
        best_path = None
        best_penalty = float('inf')
        
        for bucket, (reward, penalty, path) in pareto_sets[self.target].items():
            if penalty <= self.C and reward > best_reward:
                best_reward = reward
                best_penalty = penalty
                best_path = path
            # Break ties in favor of lower penalty
            elif penalty <= self.C and reward == best_reward and penalty < best_penalty:
                best_penalty = penalty
                best_path = path
        
        if best_path:
            return (best_reward, best_penalty, best_path)
        else:
            return None


def main():
    
    # Load the graph
    graph = load_graph_from_csv("graph_data.csv")
    
    # Set parameters
    constraint_C = 50  # Maximum allowed penalty
    epsilon = 0.1  # Approximation parameter
    
    # Run the FPTAS algorithm
    fptas = FPTAS_RRP(graph, 'n0', 'n49', constraint_C, epsilon)
    result = fptas.run()
    
    if result:
        reward, penalty, path = result
        print(f"Best path: {' -> '.join(path)}")
        print(f"Total reward: {reward}")
        print(f"Total penalty: {penalty}")
        print(f"Constraint satisfied: {penalty <= constraint_C}")
    else:
        print("No path found that satisfies the constraint.")

if __name__ == "__main__":
    main()


Best path: n0 -> n42 -> n15 -> n27 -> n10 -> n46 -> n20 -> n11 -> n29 -> n44 -> n35 -> n21 -> n23 -> n16 -> n43 -> n13 -> n47 -> n19 -> n12 -> n45 -> n33 -> n26 -> n6 -> n36 -> n4 -> n14 -> n40 -> n8 -> n37 -> n22 -> n34 -> n28 -> n32 -> n1 -> n7 -> n25 -> n5 -> n24 -> n48 -> n49
Total reward: 274
Total penalty: 2
Constraint satisfied: True
